In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/semeval-2022

/content/drive/My Drive/semeval-2022


In [ ]:
!pip install torch==1.4.0. 

In [ ]:
!pip install transformers==3.5.1

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import LineByLineTextDataset

In [ ]:
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='./maMi/mami_roberta_pretrain.txt',
    block_size=512,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:114: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
len(dataset)

5045

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:

from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./roberta-retrained-mami",
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    seed=1,
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train("./roberta-retrained-mami/checkpoint-23500")


Step,Training Loss
24000,0.155920
24500,5.492351
25000,4.867966


TrainOutput(global_step=25240, training_loss=0.3985417337311807)

In [ ]:
%reload_ext tensorboard

%tensorboard --logdir ./runs

In [ ]:
trainer.save_model("./roberta-retrained")

In [ ]:
!git clone https://github.com/pytorch/fairseq.git

Cloning into 'fairseq'...
remote: Enumerating objects: 30170, done.
remote: Counting objects: 100% (1327/1327), done.
remote: Compressing objects: 100% (846/846), done.
remote: Total 30170 (delta 662), reused 1005 (delta 460), pack-reused 28843
Receiving objects: 100% (30170/30170), 14.27 MiB | 5.52 MiB/s, done.
Resolving deltas: 100% (22337/22337), done.
Checking out files: 100% (1172/1172), done.


In [ ]:
!mkdir -p gpt2_bpe
!wget -O gpt2_bpe/encoder.json https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
!wget -O gpt2_bpe/vocab.bpe https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe

--2021-12-15 10:55:39--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘gpt2_bpe/encoder.json’

gpt2_bpe/encoder.js 100%[===================>]   1018K  3.11MB/s    in 0.3s    

2021-12-15 10:55:39 (3.11 MB/s) - ‘gpt2_bpe/encoder.json’ saved [1042301/1042301]

--2021-12-15 10:55:40--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘gpt2_bpe/vocab.bpe’

gpt2_bpe/vocab.bpe  100%[=========

In [ ]:
%cd fairseq/

/content/drive/MyDrive/semeval-2022/fairseq


In [ ]:
!pip install omegaconf

In [ ]:
!pip install hydra

     |████████████████████████████████| 82 kB 530 kB/s 
  Created wheel for hydra: filename=Hydra-2.5-cp37-cp37m-linux_x86_64.whl size=220779 sha256=f095d54dc8e4800ea53f618860a059b5a9f1bb170011d2038c539f10e60b787c
  Stored in directory: /root/.cache/pip/wheels/46/28/7d/3b38a41d900da90c4e17576f442bac9344eb1f5a4e78ee9f83
Successfully built hydra


In [ ]:
!pip install hydra.core

     |████████████████████████████████| 145 kB 5.4 MB/s 


In [ ]:
!python -m examples.roberta.multiprocessing_bpe_encoder \
        --encoder-json gpt2_bpe/encoder.json \
        --vocab-bpe gpt2_bpe/vocab.bpe \
        --inputs PCL.train.raw\
        --outputs PCL.train.bpe \
        --keep-empty \
        --workers 60; \

Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/drive/My Drive/semeval-2022/fairseq/examples/roberta/multiprocessing_bpe_encoder.py", line 14, in <module>
    from fairseq.data.encoders.gpt2_bpe import get_encoder
  File "/content/drive/My Drive/semeval-2022/fairseq/fairseq/__init__.py", line 33, in <module>
    import fairseq.criterions  # noqa
  File "/content/drive/My Drive/semeval-2022/fairseq/fairseq/criterions/__init__.py", line 24, in <module>
    "--criterion", base_class=FairseqCriterion, default="cross_entropy"
TypeError: cannot unpack non-iterable NoneType object
